# Orchestration with GenAI Hub

This notebook demonstrates, configuring an orchestration pipeline, and querying multiple LLM models with GenAI Hub.

The code imports required libraries, reads credentials from a creds.json file, and sets environment variables for authentication and API access

In [ ]:
import time
import json
import os
from IPython.display import clear_output
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
 
# Inline credentials
with open('creds.json') as f:
    credCF = json.load(f)

# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "grounding" 
    }

    for key, value in env_vars.items():
        os.environ[key] = value
        print(value)

# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )

ai_core_client = create_ai_core_client(credCF)

### Create a New Orchestration Configuration
In this step, a new configuration is created using the ai_core_client. It involves defining identifiers like scenario_id, executable_id, and a configuration name. This configuration is essential for setting up the orchestration workflow.

Key Points:

Scenario ID: Specifies the context of the orchestration scenario.

Executable ID: Identifies the executable to be used in orchestration.

In [2]:
!pip show sap-ai-sdk-gen

Name: sap-ai-sdk-gen
Version: 6.1.2
Summary: SAP Cloud SDK for AI (Python): generative AI SDK
Home-page: https://www.sap.com/
Author: SAP SE
Author-email: 
License: SAP DEVELOPER LICENSE AGREEMENT
Location: C:\Users\C5384965\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: click, dacite, h11, httpx, langchain, langchain-classic, langchain-community, langchain-openai, openai, overloading, packaging, pydantic, sap-ai-sdk-core
Required-by: 


In [ ]:
# Define scenario ID, executable ID, and configuration suffix
scenario_id = "orchestration"
executable_id = "orchestration"
config_suffix = "config-new"
config_name = f"{config_suffix}-orchestration"

# Create a new configuration
config = ai_core_client.configuration.create(
    scenario_id=scenario_id,
    executable_id=executable_id,
    name=config_name
)

print(f"Configuration created successfully with ID: {config.id} and Name: {config_name}")


### Create and Monitor the Deployment
This step involves creating a deployment using the previously created configuration ID and monitoring the deployment status until it becomes ready. The deployment is created via the ai_core_client, and a helper function (spinner) is used to check the status periodically.

Key Points:

Deployment Creation: Uses the configuration ID to create a new deployment.

Status Check: Utilizes a callback function to check if the deployment is in a 'RUNNING' state.

Spinner Function: Provides a visual indication while waiting for the deployment to be ready.

In [ ]:
# Create a deployment using the configuration ID from the previous cell
deployment = ai_core_client.deployment.create(configuration_id=config.id)

print(f"Deployment created successfully with ID: {deployment.id}")


### Monitoring the Deployment

In [4]:
from ai_api_client_sdk.models.status import Status

def spinner(check_callback, timeout=300, check_every_n_seconds=10):
    start = time.time()
    while time.time() - start < timeout:
        return_value = check_callback()
        if return_value:
            return return_value
        for char in '|/-\\':
            clear_output(wait=True)
            print(f'Waiting for the deployment to become ready... {char}')
            time.sleep(0.2)

# Define the callback to check if the deployment is ready
def check_ready():
    updated_deployment = ai_core_client.deployment.get(deployment.id)
    return updated_deployment if updated_deployment.status == Status.RUNNING else None

# Wait for the deployment to be ready
ready_deployment = spinner(check_ready)
print(f"Deployment is ready with status: {ready_deployment.status}")


Waiting for the deployment to become ready... \
Deployment is ready with status: Status.RUNNING


## Basic Orchestration Pipeline

Now that you have YOUR_DEPOLYMENT_URL, let's walk through a basic orchestration pipeline.

In [7]:
from gen_ai_hub.orchestration_v2.utils import load_text_file

# Load the CV file content
cv_file_path = "cv.txt"  # Specify the correct path to the CV file
cv_content = load_text_file(cv_file_path)

# Print the content to verify it has been loaded
print(cv_content)


John Doe
1234 Data St, San Francisco, CA 94101
(123) 456-7890
johndoe@email.com
LinkedIn Profile
GitHub Profile

Objective
Detail-oriented Data Scientist with 3+ years of experience in data analysis, statistical modeling, and machine learning. Seeking to leverage expertise in predictive modeling and data visualization to help drive data-informed decision-making at [Company Name].

Education
Master of Science in Data Science
University of California, Berkeley
Graduated: May 2021

Bachelor of Science in Computer Science
University of California, Los Angeles
Graduated: May 2019

Technical Skills

Programming Languages: Python, R, SQL, Java
Data Analysis & Visualization: Pandas, NumPy, Matplotlib, Seaborn, Tableau
Machine Learning: Scikit-learn, TensorFlow, Keras, XGBoost
Big Data Technologies: Hadoop, Spark
Databases: MySQL, PostgreSQL
Version Control: Git

Professional Experience

Data Scientist
DataCorp Inc., San Francisco, CA
June 2021 – Present

Developed predictive models to optimize

# Step 1: Templating

Explanation of Templating Code

This code defines a template for an AI assistant using orchestration configuration. The `Template` object is set up with system and user messages to guide the assistant’s response behavior. 

Key Components:
- **SystemMessage**: Sets a predefined instruction for the AI assistant. This message typically includes the assistant's role and any specific guidelines it should follow.
- **UserMessage**: Represents the user's input and how it is structured in the conversation.
  
In this revised prompt, only queries are passed to the assistant without any additional context. The AI is expected to respond based solely on the provided input.


In [ ]:
from gen_ai_hub.orchestration_v2.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration_v2.models.template import Template

# Define the template for resume screening
template = Template(
    template=[
        SystemMessage(content="""You are a helpful AI assistant for HR. Summarize the following CV in 10 sentences, 
                      focusing on key qualifications, work experience, and achievements. Include personal contact information, 
                      organizational history, and personal interests"""),
        UserMessage(content=
            "Here is a candidate's resume: {{?candidate_resume}}"
        ),
    ],
    defaults={"candidate_resume": "John Doe's resume content goes here..."},
)


# Step 2: Define the LLM 

The LLM class is used to configure and initialize a model for generating text based on specific parameters. In this example, we'll use the gpt-4o model to perform the content creation task.

ℹ️Note that virtual deployment of the model is managed automatically by the Orchestration Service, so no additional deployment setup is required on your part.

In [16]:
from gen_ai_hub.orchestration_v2.models.template import PromptTemplatingModuleConfig
from gen_ai_hub.orchestration_v2.models.llm_model_details import LLMModelDetails

llm = LLMModelDetails(
    name="gpt-5-nano",
    params={"max_completion_tokens": 2048}
)

prompt_module = PromptTemplatingModuleConfig(
    prompt=template,
    model=llm
)

This configuration initializes the model to use the llm models with the latest updates. The model will generate responses up to 256 tokens in length and produce more predictable and focused output due to the low temperature setting.

# Step 3: Create the Orchestration Configuration

The OrchestrationConfig class is used to create a configuration that integrates various components, such as templates and llm models, into a unified orchestration setup. This configuration specifies how these components work together to achieve the desired workflow.

In [17]:
from gen_ai_hub.orchestration_v2.models.config import ModuleConfig, OrchestrationConfig

modules = ModuleConfig(
    prompt_templating=prompt_module
)

config = OrchestrationConfig(modules=modules)


# Step 4: Run the Orchestration Request

The OrchestrationService class is used to interact with the orchestration service by providing a configuration and invoking its operations. This service handles the execution of workflows defined by the provided configuration and processes inputs accordingly.

In [18]:
from gen_ai_hub.orchestration_v2.service import OrchestrationService

orchestration_service = OrchestrationService(config=config)

response = orchestration_service.run(
    placeholder_values={
        "candidate_resume": cv_content
    }
)

print(response.final_result.choices[0].message.content)

John Doe, 1234 Data St, San Francisco, CA 94101; contact number (123) 456-7890; email johndoe@email.com, with LinkedIn and GitHub profiles available. 
Objective: A detail-oriented Data Scientist with 3+ years of experience in data analysis, statistical modeling, and machine learning, seeking to leverage predictive modeling and data visualization to drive data-informed decision-making at your company. 
Education: Master of Science in Data Science from the University of California, Berkeley (May 2021) and Bachelor of Science in Computer Science from the University of California, Los Angeles (May 2019). 
Technical Skills: Python, R, SQL, Java; data analysis and visualization with Pandas, NumPy, Matplotlib, Seaborn, Tableau; machine learning with Scikit-learn, TensorFlow, Keras, XGBoost; big data technologies Hadoop and Spark; databases MySQL and PostgreSQL; version control with Git. 
Professional Experience: Data Scientist at DataCorp Inc., San Francisco, CA (June 2021 – Present), develop